In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

hospital_df = pd.read_csv('../downloads/normalizacion/hospital.csv', delimiter=',')
hospital_activity_df = pd.read_csv('../downloads/normalizacion/hospital_activity.csv', delimiter=';')
hospital_municipality_df = pd.read_csv('../downloads/normalizacion/hospital_municipality.csv', delimiter=';')
hospital_resources_df = pd.read_csv('../downloads/normalizacion/hospital_resources.csv', delimiter=';')
municipality_df = pd.read_csv('../downloads/normalizacion/municipality.csv', delimiter=',')
municipality_demographics_df = pd.read_csv('../downloads/normalizacion/municipality_demographics.csv', delimiter=',')

# Filtro solicitado sobre hospital_municipality_df
exclude_hospital_id = 'hospital_central_de_la_defensa_gomez_ulla'
exclude_municipality_names = ['Moratalaz', 'Vicálvaro y Retiro', 'Braojos de la Sierra', 'nan']

# Convertir municipality_id a string para evitar problemas de comparación
hospital_municipality_df['municipality_id'] = hospital_municipality_df['municipality_id'].astype(str)

# Filtrar los municipality_id nulos y vacíos
hospital_municipality_df = hospital_municipality_df[hospital_municipality_df['municipality_id'].notnull() & (hospital_municipality_df['municipality_id'] != '')]

# Filtrar por municipality_id
hospital_municipality_df = hospital_municipality_df[~hospital_municipality_df['municipality_id'].isin(exclude_municipality_names)]

# Filtrar por hospital_id
hospital_municipality_df = hospital_municipality_df[hospital_municipality_df['hospital_id'] != exclude_hospital_id]

hospital_resources_2022 = hospital_resources_df[hospital_resources_df['year'].astype(str) == '2022'].copy()
hospital_resources_2022['hospital_id'] = hospital_resources_2022['hospital_id'].astype(str)

hospital_activity_2022 = hospital_activity_df[hospital_activity_df['year'].astype(str) == '2022'].copy()
hospital_activity_2022['hospital_id'] = hospital_activity_2022['hospital_id'].astype(str)

municipality_demographics_2022 = municipality_demographics_df[municipality_demographics_df['year'].astype(str) == '2022'].copy()
municipality_demographics_2022['id_secondary_municipality'] = municipality_demographics_2022['id_secondary_municipality'].astype(str)


In [2]:
# === CÁLCULO DEL NIVEL DE SERVICIO DE HOSPITALES ===

# 1. PREPARAR DATOS DE RECURSOS POR HOSPITAL
hospital_resources_pivot = hospital_resources_2022.pivot_table(
    index='hospital_id',
    columns='type_resources',
    values='total',
    fill_value=0
).reset_index()

# 2. PREPARAR DATOS DE ACTIVIDAD POR HOSPITAL
hospital_activity_pivot = hospital_activity_2022.pivot_table(
    index='hospital_id',
    columns='type_activity',
    values='total',
    fill_value=0
).reset_index()

# 3. UNIR RECURSOS Y ACTIVIDAD POR HOSPITAL
hospital_service_df = hospital_resources_pivot.merge(hospital_activity_pivot, on='hospital_id', how='outer')

# Renombrar columnas a snake_case
column_mapping = {
    'Camas instaladas': 'camas_instaladas',
    'Estancia media global': 'estancia_media_global',
    'Ingresos programados': 'ingresos_programados',
    'Ingresos urgentes': 'ingresos_urgentes',
    'Total ingresos': 'total_ingresos',
    'Urgencias totales': 'urgencias_totales'
}
hospital_service_df = hospital_service_df.rename(columns=column_mapping)

# 4. CALCULAR INDICADORES DE NIVEL DE SERVICIO

# 4.1 Tasa de ocupación de camas (días de hospitalización / (camas * 365))
if 'camas_instaladas' in hospital_service_df.columns and 'total_ingresos' in hospital_service_df.columns and 'estancia_media_global' in hospital_service_df.columns:
    hospital_service_df['dias_hospitalizacion'] = hospital_service_df['total_ingresos'] * hospital_service_df['estancia_media_global']
    hospital_service_df['capacidad_anual_camas'] = hospital_service_df['camas_instaladas'] * 365
    hospital_service_df['tasa_ocupacion_camas'] = np.where(
        hospital_service_df['capacidad_anual_camas'] > 0,
        hospital_service_df['dias_hospitalizacion'] / hospital_service_df['capacidad_anual_camas'],
        0
    )

# 4.2 Eficiencia en urgencias (% de ingresos desde urgencias)
if 'urgencias_totales' in hospital_service_df.columns and 'total_ingresos' in hospital_service_df.columns:
    hospital_service_df['tasa_ingreso_urgencias'] = np.where(
        hospital_service_df['urgencias_totales'] > 0,
        hospital_service_df['total_ingresos'] / hospital_service_df['urgencias_totales'],
        0
    )

# 4.3 Eficiencia de ingresos urgentes vs programados
if 'ingresos_urgentes' in hospital_service_df.columns and 'ingresos_programados' in hospital_service_df.columns:
    hospital_service_df['total_ingresos_calculado'] = hospital_service_df['ingresos_urgentes'] + hospital_service_df['ingresos_programados']
    hospital_service_df['ratio_urgentes_programados'] = np.where(
        hospital_service_df['ingresos_programados'] > 0,
        hospital_service_df['ingresos_urgentes'] / hospital_service_df['ingresos_programados'],
        0
    )

# 4.4 Productividad de camas (ingresos por cama)
if 'total_ingresos' in hospital_service_df.columns and 'camas_instaladas' in hospital_service_df.columns:
    hospital_service_df['productividad_camas'] = np.where(
        hospital_service_df['camas_instaladas'] > 0,
        hospital_service_df['total_ingresos'] / hospital_service_df['camas_instaladas'],
        0
    )

# 4.5 Eficiencia de estancia (menor estancia = más eficiente, pero normalizado)
if 'estancia_media_global' in hospital_service_df.columns:
    hospital_service_df['eficiencia_estancia'] = np.where(
        hospital_service_df['estancia_media_global'] > 0,
        1 / hospital_service_df['estancia_media_global'],  # Invertir para que menor estancia = mayor eficiencia
        0
    )

# 5. CALCULAR ÍNDICE COMPUESTO DE NIVEL DE SERVICIO

# Actualizar indicadores disponibles
service_indicators = ['tasa_ocupacion_camas', 'tasa_ingreso_urgencias', 'ratio_urgentes_programados',
                     'productividad_camas', 'eficiencia_estancia']

# Crear copia para normalización
hospital_service_normalized = hospital_service_df.copy()

for indicator in service_indicators:
    if indicator in hospital_service_normalized.columns:
        # Usar percentiles para normalizar (0-100)
        hospital_service_normalized[f'{indicator}_percentile'] = hospital_service_normalized[indicator].rank(pct=True) * 100

        # Para tasa de ocupación, el óptimo está cerca del 80-85%, no al máximo
        if indicator == 'tasa_ocupacion_camas':
            # Penalizar ocupaciones muy bajas (<50%) y muy altas (>95%)
            hospital_service_normalized[f'{indicator}_score'] = np.where(
                hospital_service_normalized[indicator] < 0.5,
                hospital_service_normalized[indicator] * 2 * 100,  # Escala 0-50 para <50%
                np.where(
                    hospital_service_normalized[indicator] > 0.95,
                    100 - (hospital_service_normalized[indicator] - 0.95) * 1000,  # Penalizar >95%
                    100  # Óptimo entre 50-95%
                )
            )
        else:
            hospital_service_normalized[f'{indicator}_score'] = hospital_service_normalized[f'{indicator}_percentile']

# Calcular índice compuesto de nivel de servicio (promedio de scores disponibles)
score_columns = [col for col in hospital_service_normalized.columns if col.endswith('_score')]
if score_columns:
    hospital_service_normalized['nivel_servicio_score'] = hospital_service_normalized[score_columns].mean(axis=1, skipna=True)
else:
    hospital_service_normalized['nivel_servicio_score'] = 0

# 6. AGREGAR INFORMACIÓN DE MUNICIPIO Y NOMBRES DESCRIPTIVOS (SOLO HOSPITALES CON MUNICIPIO)
hospital_service_final = hospital_service_normalized.merge(
    hospital_municipality_df[['hospital_id', 'municipality_id']],
    on='hospital_id',
    how='inner'  # Cambio a INNER JOIN - solo hospitales con municipio asignado
)

# 6.1 Agregar nombres de hospitales
hospital_df['id'] = hospital_df['id'].astype(str)
hospital_service_final = hospital_service_final.merge(
    hospital_df[['id', 'name']].rename(columns={'id': 'hospital_id', 'name': 'hospital_name'}),
    on='hospital_id',
    how='inner'  # INNER JOIN para asegurar que tenemos nombres
)

# 6.2 Agregar nombres de municipios y datos demográficos
municipality_df['id'] = municipality_df['id'].astype(str)
hospital_service_final = hospital_service_final.merge(
    municipality_df[['id', 'name', 'id_secondary']].rename(columns={'id': 'municipality_id', 'name': 'municipality_name'}),
    on='municipality_id',
    how='inner'  # INNER JOIN para asegurar que tenemos nombres de municipio
)


In [3]:
# === ANÁLISIS DE POBLACIÓN ASIGNADA A HOSPITALES ===

# 6.3 Preparar datos demográficos (población total por municipio)
population_data = municipality_demographics_2022[
    (municipality_demographics_2022['range'] == 'total')
].copy()

# 6.4 Agregar población total del municipio a cada hospital
# Asegurar que los tipos de datos sean compatibles para el merge
municipality_df['id_secondary'] = municipality_df['id_secondary'].astype(str)
population_data['id_secondary_municipality'] = population_data['id_secondary_municipality'].astype(str)

# Hacer el merge con manejo de errores
try:
    hospital_service_final = hospital_service_final.merge(
        population_data[['id_secondary_municipality', 'total']].rename(columns={
            'id_secondary_municipality': 'id_secondary',
            'total': 'poblacion_total_municipio'
        }),
        on='id_secondary',
        how='left'
    )

except Exception as e:
    # Método alternativo: crear el merge paso a paso
    population_renamed = population_data[['id_secondary_municipality', 'total']].copy()
    population_renamed = population_renamed.rename(columns={
        'id_secondary_municipality': 'id_secondary',
        'total': 'poblacion_total_municipio'
    })

    # Asegurar tipos compatibles
    population_renamed['id_secondary'] = population_renamed['id_secondary'].astype(str)
    hospital_service_final['id_secondary'] = hospital_service_final['id_secondary'].astype(str)

    # Intentar merge nuevamente
    hospital_service_final = hospital_service_final.merge(
        population_renamed,
        on='id_secondary',
        how='left'
    )

# Rellenar valores nulos de población con 0
hospital_service_final['poblacion_total_municipio'] = hospital_service_final['poblacion_total_municipio'].fillna(0)


In [4]:
# === CÁLCULO DE INDICADORES DE CAPACIDAD POR POBLACIÓN ===

# 6.5 Calcular indicadores de capacidad por población
if 'camas_instaladas' in hospital_service_final.columns:
    # Camas por cada 1000 habitantes
    hospital_service_final['camas_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['camas_instaladas'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

if 'total_ingresos' in hospital_service_final.columns:
    # Ingresos hospitalarios por cada 1000 habitantes
    hospital_service_final['ingresos_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['total_ingresos'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

if 'urgencias_totales' in hospital_service_final.columns:
    # Urgencias por cada 1000 habitantes
    hospital_service_final['urgencias_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['urgencias_totales'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

# 6.6 Calcular población total asignada a cada hospital (suma de todos sus municipios)
# Obtener todos los municipios asignados a cada hospital
hospital_municipalities_population = hospital_service_final.groupby('hospital_id').agg({
    'poblacion_total_municipio': 'sum',  # Suma total de población de todos los municipios asignados
    'municipality_name': lambda x: ', '.join(x.unique()),  # Lista de municipios asignados
    'municipality_id': 'count'  # Número de municipios asignados
}).rename(columns={
    'poblacion_total_municipio': 'poblacion_total_asignada',
    'municipality_name': 'municipios_asignados',
    'municipality_id': 'num_municipios_asignados'
}).reset_index()

# Unir esta información de vuelta al DataFrame principal
hospital_service_final = hospital_service_final.drop(columns=['poblacion_estimada_servida'], errors='ignore')
hospital_service_final = hospital_service_final.merge(
    hospital_municipalities_population[['hospital_id', 'poblacion_total_asignada', 'num_municipios_asignados']],
    on='hospital_id',
    how='left'
)

# 6.7 Indicadores de preparación hospitalaria para población total asignada
if 'camas_instaladas' in hospital_service_final.columns:
    # Camas por cada 1000 habitantes de la población total asignada al hospital
    hospital_service_final['camas_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['camas_instaladas'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

    # Clasificación de preparación según estándares internacionales (2-4 camas por 1000 hab)
    hospital_service_final['preparacion_camas'] = np.where(
        hospital_service_final['camas_por_1000_asignados'] >= 4, 'Excelente',
        np.where(hospital_service_final['camas_por_1000_asignados'] >= 2, 'Adecuada',
                np.where(hospital_service_final['camas_por_1000_asignados'] >= 1, 'Básica', 'Insuficiente'))
    )

if 'total_ingresos' in hospital_service_final.columns:
    # Capacidad de atención por población total asignada
    hospital_service_final['capacidad_atencion_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['total_ingresos'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

if 'urgencias_totales' in hospital_service_final.columns:
    # Urgencias por cada 1000 habitantes de población total asignada
    hospital_service_final['urgencias_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['urgencias_totales'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

# 6.8 Índice compuesto de preparación hospitalaria (actualizado)
preparacion_indicators = []
if 'camas_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('camas_por_1000_asignados')
if 'capacidad_atencion_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('capacidad_atencion_por_1000_asignados')
if 'urgencias_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('urgencias_por_1000_asignados')

# Normalizar indicadores de preparación usando percentiles
for indicator in preparacion_indicators:
    if indicator in hospital_service_final.columns:
        hospital_service_final[f'{indicator}_percentile'] = hospital_service_final[indicator].rank(pct=True) * 100

# Calcular índice de preparación poblacional actualizado
if preparacion_indicators:
    percentile_cols = [f'{ind}_percentile' for ind in preparacion_indicators if f'{ind}_percentile' in hospital_service_final.columns]
    if percentile_cols:
        hospital_service_final['indice_preparacion_poblacional'] = hospital_service_final[percentile_cols].mean(axis=1, skipna=True)
    else:
        hospital_service_final['indice_preparacion_poblacional'] = 0
else:
    hospital_service_final['indice_preparacion_poblacional'] = 0

# Eliminar duplicados por hospital para mostrar resultados únicos
hospital_unique_results = hospital_service_final.drop_duplicates(subset=['hospital_id'])


In [5]:
# === MODIFICAR HOSPITAL_UNIQUE_RESULTS ===

# Crear un DataFrame con todos los municipios por hospital
hospital_municipalities_list = hospital_service_final.groupby('hospital_id').agg({
    'municipality_name': lambda x: list(x.unique()),  # Lista de nombres de municipios únicos
    'municipality_id': lambda x: list(x.unique())     # Lista de IDs de municipios únicos
}).rename(columns={
    'municipality_name': 'municipios_servidos',
    'municipality_id': 'municipios_id_servidos'
}).reset_index()

# Hacer merge con hospital_unique_results para agregar la lista de municipios
hospital_unique_results = hospital_unique_results.merge(
    hospital_municipalities_list,
    on='hospital_id',
    how='left'
)

# Eliminar las columnas especificadas
columns_to_remove = ['municipality_name', 'id_secondary', 'poblacion_total_municipio', 'municipality_id']
hospital_unique_results = hospital_unique_results.drop(columns=columns_to_remove, errors='ignore')

# Imprimir las columnas de hospital_unique_results
print("="*80)
print("📊 COLUMNAS DEL DATAFRAME: hospital_unique_results")
print("="*80)
print(f"Total de columnas: {len(hospital_unique_results.columns)}")
print("\nListado de columnas:")
for i, col in enumerate(hospital_unique_results.columns, 1):
    print(f"{i:2d}. {col}")


📊 COLUMNAS DEL DATAFRAME: hospital_unique_results
Total de columnas: 42

Listado de columnas:
 1. hospital_id
 2. camas_instaladas
 3. estancia_media_global
 4. ingresos_programados
 5. ingresos_urgentes
 6. total_ingresos
 7. urgencias_totales
 8. dias_hospitalizacion
 9. capacidad_anual_camas
10. tasa_ocupacion_camas
11. tasa_ingreso_urgencias
12. total_ingresos_calculado
13. ratio_urgentes_programados
14. productividad_camas
15. eficiencia_estancia
16. tasa_ocupacion_camas_percentile
17. tasa_ocupacion_camas_score
18. tasa_ingreso_urgencias_percentile
19. tasa_ingreso_urgencias_score
20. ratio_urgentes_programados_percentile
21. ratio_urgentes_programados_score
22. productividad_camas_percentile
23. productividad_camas_score
24. eficiencia_estancia_percentile
25. eficiencia_estancia_score
26. nivel_servicio_score
27. hospital_name
28. camas_por_1000_hab
29. ingresos_por_1000_hab
30. urgencias_por_1000_hab
31. poblacion_total_asignada
32. num_municipios_asignados
33. camas_por_1000_a

In [6]:
# === GUARDAR HOSPITAL_UNIQUE_RESULTS COMO CSV ===

# Crear directorio si no existe
output_dir = '../downloads/normalizacion/'
os.makedirs(output_dir, exist_ok=True)

# Guardar el DataFrame hospital_unique_results
hospital_file = os.path.join(output_dir, 'hospital_analysis.csv')
hospital_unique_results.to_csv(hospital_file, index=False, encoding='utf-8')

print("\n" + "="*80)
print("💾 ARCHIVO CSV GENERADO EXITOSAMENTE")
print("="*80)
print(f"✅ Archivo guardado: {hospital_file}")
print(f"   Registros: {len(hospital_unique_results)}")
print(f"   Columnas: {len(hospital_unique_results.columns)}")
print(f"   Encoding: UTF-8")
print("="*80)



💾 ARCHIVO CSV GENERADO EXITOSAMENTE
✅ Archivo guardado: ../downloads/normalizacion/hospital_analysis.csv
   Registros: 18
   Columnas: 42
   Encoding: UTF-8
